In [1]:
import flask, flask_restful, flask_uploads, flask_cors, boto3, http, os, json, dotenv
from resources import generate_json

dotenv.load_dotenv()

attachmentSet = flask_uploads.UploadSet("files", ("csv",))
s3Bucket = boto3.resource(
    "s3",
    region_name=os.environ.get("AWS_REGION"),
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_KEY")
)

In [2]:
class UploadResource(flask_restful.Resource):
    def post(self):
        if flask.request.method == 'OPTIONS':
            response = flask.make_response()
            response.headers.add("Access-Control-Allow-Origin", "*")
            response.headers.add('Access-Control-Allow-Headers', "*")
            response.headers.add('Access-Control-Allow-Methods', "*")
            return response
        elif flask.request.method == 'POST' and "csvData" in flask.request.files:
            try:
                # receive csv file from frontend
                file = flask.request.files['csvData']
                attachmentSet.save(file, folder="input_storage", name=file.filename)
                attachmentPath = attachmentSet.path(folder="input_storage", name=file.filename)

                # generate json data
                localPath, fileName = generate_json(file_csv=attachmentPath, short=True)

                # upload json file to s3 bucket and send it back to frontend
                s3Bucket.Bucket(os.environ.get("PROJECT_BUCKET")).upload_file(
                    Filename=localPath,
                    Key=f"filesCollection/{fileName}",
                    ExtraArgs={'ACL': 'public-read'}
                )

                os.remove(localPath)
                os.remove(attachmentPath)

                contentObject = s3Bucket.Object(os.environ.get("PROJECT_BUCKET"), f"{fileName}")
                jsonContent = json.loads(contentObject.get()['Body'].read().decode('utf-8'))
                return jsonContent["Details"], http.HTTPStatus.ACCEPTED

            except flask_uploads.UploadNotAllowed or Exception as e:
                if flask_uploads.UploadNotAllowed:
                    return {"message": "Only CSV files"}, http.HTTP.BAD_REQUEST
                elif e:
                    return {"message": e}, http.HTTPStatus.BAD_REQUEST

In [3]:
def create_app():
    app = flask.Flask(__name__)
    flask_cors.CORS(app, expose_headers='Authorization')
    app.config['UPLOADED_FILES_DEST'] = 'static/local_storage'
    flask_uploads.configure_uploads(app, attachmentSet)

    @app.route('/')
    def index():
        return {'message': 'root api'}

    register_resources(app)
    return app

In [4]:
def register_resources(app):
    api = flask_restful.Api(app)
    api.add_resource(UploadResource, '/upload_csv')

In [5]:
app = create_app()
app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


2020-09-15 20:31:04 [INFO] [PID:59492 TID:4335746496] [_internal.py:122 in `_log`]   * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
2020-09-15 20:31:04 [INFO] [PID:59492 TID:4335746496] [_internal.py:122 in `_log`]   * Restarting with fsevents reloader


SystemExit: 1

/Users/timhuynh0905/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
